In [37]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

In [38]:
plays = pd.read_csv('plays.csv')
players = pd.read_csv('players.csv')
playerplay = pd.read_csv('player_play.csv')
df = pd.read_csv('tracking_week_1.csv')

In [39]:
blockers = playerplay[['gameId','playId','nflId','blockedPlayerNFLId1','blockedPlayerNFLId2','blockedPlayerNFLId3','pressureAllowedAsBlocker','timeToPressureAllowedAsBlocker']][playerplay['blockedPlayerNFLId1'].notna()]

In [40]:
blockers = pd.merge(
    blockers,
    players[['nflId','position', 'height','weight']],
    how = "left",
    left_on = 'nflId',
    right_on = 'nflId'
)

In [41]:
blockers = pd.merge(
    blockers,
    playerplay[['gameId','playId','nflId','wasInitialPassRusher','causedPressure','timeToPressureAsPassRusher','getOffTimeAsPassRusher']],
    how = "left",
    left_on = ['gameId','playId','blockedPlayerNFLId1'],
    right_on = ['gameId','playId','nflId']
)

In [42]:
ballsnap = df[df['frameType']=="SNAP"]

In [43]:
weekoneblockers = pd.merge(
    blockers,
    ballsnap[['gameId','playId','nflId','x','y']],
    how = "inner",
    left_on = ['gameId','playId','nflId_x'],
    right_on = ['gameId','playId','nflId']
)

In [44]:
weekoneblockers.drop(columns = ['nflId_x','nflId_y'], inplace=True)

In [45]:
weekoneblockers = pd.merge(
    weekoneblockers,
    ballsnap[['gameId','playId','nflId','x','y']],
    how = "inner",
    left_on = ['gameId','playId','blockedPlayerNFLId1'],
    right_on = ['gameId','playId','nflId']
)

In [46]:
weekoneblockers = pd.merge(
    weekoneblockers,
    players[['nflId','position','height','weight']],
    how = "left",
    left_on = 'blockedPlayerNFLId1',
    right_on = 'nflId'
)

In [48]:
weekoneblockers = weekoneblockers[['gameId','playId','nflId_x','x_x','y_x','position_x','height_x','weight_x','blockedPlayerNFLId1','blockedPlayerNFLId2','blockedPlayerNFLId3','x_y','y_y','position_y','height_y','weight_y','pressureAllowedAsBlocker','timeToPressureAllowedAsBlocker','wasInitialPassRusher','getOffTimeAsPassRusher']]
weekoneblockers

,gameId,playId,nflId_x,x_x,y_x,position_x,height_x,weight_x,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,x_y,y_y,position_y,height_y,weight_y,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,wasInitialPassRusher,getOffTimeAsPassRusher
0,2022090800,56,35472.0,86.09,28.14,G,6-5,325,47917.0,NaN,NaN,83.99,29.69,NT,6-1,312,0.0,NaN,1.0,1.284
1,2022090800,56,42392.0,85.57,29.67,C,6-6,305,47917.0,NaN,NaN,83.99,29.69,NT,6-1,312,0.0,NaN,1.0,1.284
2,2022090800,56,44875.0,85.90,26.35,T,6-5,320,43335.0,NaN,NaN,83.91,26.37,DT,6-4,330,0.0,NaN,1.0,1.384
3,2022090800,56,48512.0,86.16,31.22,C,6-4,300,41239.0,NaN,NaN,84.16,32.42,DT,6-1,280,0.0,NaN,1.0,1.084
4,2022090800,56,53522.0,86.13,32.87,T,6-9,321,43298.0,NaN,NaN,84.23,36.10,DE,6-5,240,0.0,NaN,1.0,0.984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6140,2022091200,3826,41369.0,65.99,32.84,T,6-6,320,52456.0,NaN,NaN,63.78,37.90,OLB,6-3,253,0.0,NaN,1.0,0.935
6141,2022091200,3826,43384.0,65.74,31.43,G,6-6,310,43436.0,NaN,NaN,64.15,32.71,DE,6-4,291,0.0,NaN,1.0,0.935
6142,2022091200,3826,52491.0,65.10,29.88,C,6-4,315,43436.0,NaN,NaN,64.15,32.71,DE,6-4,291,0.0,NaN,1.0,0.935
6143,2022091200,3826,44832.0,66.32,26.67,T,6-5,300,41464.0,54505.0,NaN,63.94,27.59,DT,6-2,290,0.0,NaN,1.0,1.035


In [50]:
weekoneblockers.rename(columns={
    'nflId_x': 'blockerId',
    'x_x': 'blockerX',
    'y_x': 'blockerY',
    'position_x': 'blockerPosition',
    'position_y': 'rusherPosition',
    'height_x': 'blockerHeight',
    'weight_x': 'blockerWeight',
    'height_y': 'rusherHeight',
    'weight_y': 'rusherWeight',
    'x_y': 'rusherX',
    'y_y': 'rusherY'
    }, inplace=True)

C:\Users\bhaga\AppData\Local\Temp\ipykernel_19488\2531081866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekoneblockers.rename(columns={


In [55]:
def height_to_inches(height):
    feet, inches = map(int, height.split('-'))  # Split the string and convert to integers
    return feet * 12 + inches

In [56]:
weekoneblockers['blockerHeight'] = weekoneblockers['blockerHeight'].apply(height_to_inches)
weekoneblockers['rusherHeight'] = weekoneblockers['rusherHeight'].apply(height_to_inches)

C:\Users\bhaga\AppData\Local\Temp\ipykernel_19488\2210431182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekoneblockers['blockerHeight'] = weekoneblockers['blockerHeight'].apply(height_to_inches)
C:\Users\bhaga\AppData\Local\Temp\ipykernel_19488\2210431182.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekoneblockers['rusherHeight'] = weekoneblockers['rusherHeight'].apply(height_to_inches)


In [59]:
weekoneblockers['hieghtDifference'] = weekoneblockers['blockerHeight'] - weekoneblockers['rusherHeight']
weekoneblockers['weightDifference'] = weekoneblockers['blockerWeight'] - weekoneblockers['rusherWeight']
weekoneblockers.drop(columns=['blockerWeight','blockerHeight','rusherWeight','rusherHeight'],inplace=True)
weekoneblockers

C:\Users\bhaga\AppData\Local\Temp\ipykernel_19488\2282890102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekoneblockers['hieghtDifference'] = weekoneblockers['blockerHeight'] - weekoneblockers['rusherHeight']
C:\Users\bhaga\AppData\Local\Temp\ipykernel_19488\2282890102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekoneblockers['weightDifference'] = weekoneblockers['blockerWeight'] - weekoneblockers['rusherWeight']
C:\Users\bhaga\AppData\Local\Temp\ipykernel_19488\2282890102.py:3: SettingWi

,gameId,playId,blockerId,blockerX,blockerY,blockerPosition,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,rusherX,rusherY,rusherPosition,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,wasInitialPassRusher,getOffTimeAsPassRusher,hieghtDifference,weightDifference
0,2022090800,56,35472.0,86.09,28.14,G,47917.0,NaN,NaN,83.99,29.69,NT,0.0,NaN,1.0,1.284,4,13
1,2022090800,56,42392.0,85.57,29.67,C,47917.0,NaN,NaN,83.99,29.69,NT,0.0,NaN,1.0,1.284,5,-7
2,2022090800,56,44875.0,85.90,26.35,T,43335.0,NaN,NaN,83.91,26.37,DT,0.0,NaN,1.0,1.384,1,-10
3,2022090800,56,48512.0,86.16,31.22,C,41239.0,NaN,NaN,84.16,32.42,DT,0.0,NaN,1.0,1.084,3,20
4,2022090800,56,53522.0,86.13,32.87,T,43298.0,NaN,NaN,84.23,36.10,DE,0.0,NaN,1.0,0.984,4,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6140,2022091200,3826,41369.0,65.99,32.84,T,52456.0,NaN,NaN,63.78,37.90,OLB,0.0,NaN,1.0,0.935,3,67
6141,2022091200,3826,43384.0,65.74,31.43,G,43436.0,NaN,NaN,64.15,32.71,DE,0.0,NaN,1.0,0.935,2,19
6142,2022091200,3826,52491.0,65.10,29.88,C,43436.0,NaN,NaN,64.15,32.71,DE,0.0,NaN,1.0,0.935,0,24
6143,2022091200,3826,44832.0,66.32,26.67,T,41464.0,54505.0,NaN,63.94,27.59,DT,0.0,NaN,1.0,1.035,3,10


In [78]:
weekoneblockers = pd.get_dummies(weekoneblockers, columns=['blockerPosition','rusherPosition'])

In [80]:
weekoneblockers.columns

Index(['gameId', 'playId', 'blockerId', 'blockerX', 'blockerY',
       'blockedPlayerNFLId1', 'blockedPlayerNFLId2', 'blockedPlayerNFLId3',
       'rusherX', 'rusherY', 'pressureAllowedAsBlocker',
       'timeToPressureAllowedAsBlocker', 'wasInitialPassRusher',
       'getOffTimeAsPassRusher', 'hieghtDifference', 'weightDifference',
       'blockerPosition_C', 'blockerPosition_FB', 'blockerPosition_G',
       'blockerPosition_RB', 'blockerPosition_T', 'blockerPosition_TE',
       'blockerPosition_WR', 'rusherPosition_CB', 'rusherPosition_DB',
       'rusherPosition_DE', 'rusherPosition_DT', 'rusherPosition_FS',
       'rusherPosition_ILB', 'rusherPosition_LB', 'rusherPosition_MLB',
       'rusherPosition_NT', 'rusherPosition_OLB', 'rusherPosition_SS'],
      dtype='object')

In [62]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


In [166]:
selected_features = ['blockerX','blockerY','rusherX','rusherY','getOffTimeAsPassRusher','hieghtDifference','weightDifference', 'blockerPosition_C', 'blockerPosition_FB', 'blockerPosition_G',
       'blockerPosition_RB', 'blockerPosition_T', 'blockerPosition_TE',
       'blockerPosition_WR', 'rusherPosition_CB', 'rusherPosition_DB',
       'rusherPosition_DE', 'rusherPosition_DT', 'rusherPosition_FS',
       'rusherPosition_ILB', 'rusherPosition_LB', 'rusherPosition_MLB',
       'rusherPosition_NT', 'rusherPosition_OLB', 'rusherPosition_SS']

In [167]:
for index, row in weekoneblockers.iterrows():
    play = weekoneblockers[
        (weekoneblockers['gameId'] == row['gameId']) &
        (weekoneblockers['playId'] == row['playId'])
    ]
    if (play['pressureAllowedAsBlocker'] == 1).any():
        weekoneblockers.loc[index, 'playPressure'] = 1
    else:
        weekoneblockers.loc[index, 'playPressure'] = 0


In [177]:
weekoneblockers[selected_features].dtypes

blockerX                  float64
blockerY                  float64
rusherX                   float64
rusherY                   float64
getOffTimeAsPassRusher    float64
hieghtDifference            int64
weightDifference            int64
blockerPosition_C            bool
blockerPosition_FB           bool
blockerPosition_G            bool
blockerPosition_RB           bool
blockerPosition_T            bool
blockerPosition_TE           bool
blockerPosition_WR           bool
rusherPosition_CB            bool
rusherPosition_DB            bool
rusherPosition_DE            bool
rusherPosition_DT            bool
rusherPosition_FS            bool
rusherPosition_ILB           bool
rusherPosition_LB            bool
rusherPosition_MLB           bool
rusherPosition_NT            bool
rusherPosition_OLB           bool
rusherPosition_SS            bool
dtype: object

In [178]:
class NFLPlayDataset(Dataset):
    def __init__(self, dataframe, selected_features, label_column):
        """
        Args:
            dataframe (pd.DataFrame): Input dataset.
            selected_features (list): Columns to use as features.
            label_column (str): Column containing the target variable.
        """
        self.dataframe = weekoneblockers
        self.selected_features = selected_features
        self.label_column = 'playPressure'
        self.groups = dataframe.groupby(['gameId', 'playId'])

        # Extract grouped sequences and labels
        self.sequences = []
        self.labels = []
        for _, group in self.groups:
            self.sequences.append(group[selected_features].values.astype(np.float32))
            self.labels.append(group[label_column].iloc[0])  # Same label for entire play

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)


In [201]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

def collate_fn(batch):
    inputs, labels = zip(*batch)

    # Set the maximum sequence length to 7
    max_len = 7

    # Pad sequences to the maximum length
    padded_inputs = [torch.nn.functional.pad(x, (0, max_len - len(x))) for x in inputs]

    # Convert padded sequences to a tensor
    padded_inputs = torch.stack(padded_inputs, dim=0)

    labels = torch.tensor(labels)

    return padded_inputs, labels

In [ ]:
# Example usage with your dataset
df = weekoneblockers

# Define features and label
label_column = 'pressureAllowedAsBlocker'  # Replace with your actual label column

# Split into train/test
train_df = df[df['gameId'] < 2022091111]  # Example train split
test_df = df[df['gameId'] >= 2022091111]  # Example test split

# Create datasets
train_dataset = NFLPlayDataset(train_df, selected_features, label_column)
test_dataset = NFLPlayDataset(test_df, selected_features, label_column)

In [281]:
modified_train_dataset = []  # Create a new list to store the modified data

for i in train_dataset:
    # Get the tensors from the first element of i (assuming it's a tuple of tensors)
    tensors = list(i[0])  # Ensure tensors are in a list
    num_tensors = len(tensors)
    
    if num_tensors < 9:
        # Create a list of zero tensors to pad
        num_to_add = 9 - num_tensors
        empty_tensor = torch.zeros(25)  # Create an empty tensor of size 25
        tensors.extend([empty_tensor] * num_to_add)  # Add padding tensors
    
    # Stack the tensors into a single tensor of shape (7, 25)
    combined_tensor = torch.stack(tensors)

    # Reconstruct the tuple with the combined tensor and other elements
    new_tuple = (combined_tensor, *i[1:])
    modified_train_dataset.append(new_tuple)  # Append the modified tuple to the new dataset


In [282]:
modified_test_dataset = []  # Create a new list to store the modified data

for i in test_dataset:
    # Get the tensors from the first element of i (assuming it's a tuple of tensors)
    tensors = list(i[0])  # Ensure tensors are in a list
    num_tensors = len(tensors)
    
    if num_tensors < 9:
        # Create a list of zero tensors to pad
        num_to_add = 9 - num_tensors
        empty_tensor = torch.zeros(25)  # Create an empty tensor of size 25
        tensors.extend([empty_tensor] * num_to_add)  # Add padding tensors
    
    # Stack the tensors into a single tensor of shape (7, 25)
    combined_tensor = torch.stack(tensors)

    # Reconstruct the tuple with the combined tensor and other elements
    new_tuple = (combined_tensor, *i[1:])
    modified_test_dataset.append(new_tuple)  # Append the modified tuple to the new dataset


In [283]:
train_loader = DataLoader(modified_train_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(modified_test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [185]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define RNN Model
class PlayRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(PlayRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: PackedSequence of [batch_size, sequence_length, input_size]
        packed_input, lengths = x  # Get the packed input and sequence lengths
        
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, packed_input.batch_sizes[0], self.hidden_size).to(packed_input.data.device)
        c0 = torch.zeros(self.num_layers, packed_input.batch_sizes[0], self.hidden_size).to(packed_input.data.device)

        # Pass packed input through LSTM
        packed_out, _ = self.rnn(packed_input, (h0, c0))

        # Unpack the output
        out, _ = pad_packed_sequence(packed_out, batch_first=True)

        # Use the last hidden state (last valid time step)
        out = out[torch.arange(out.size(0)), lengths - 1, :]  # Take the last valid element
        out = self.fc(out)
        return out


In [76]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [296]:
for i in train_loader:
    print(i)

[tensor([[[86.0900, 28.1400, 83.9900,  ...,  1.0000,  0.0000,  0.0000],
         [85.5700, 29.6700, 83.9900,  ...,  1.0000,  0.0000,  0.0000],
         [85.9000, 26.3500, 83.9100,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[80.3500, 28.0000, 78.0800,  ...,  1.0000,  0.0000,  0.0000],
         [79.7000, 29.4500, 78.0800,  ...,  1.0000,  0.0000,  0.0000],
         [79.9900, 25.8800, 78.0300,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[66.5400, 27.8800, 64.7200,  ...,  0.0000,  0.0000,  0.0000],
         [65.8400, 29.4100, 64.1800,  ...,  

In [295]:
# Hyperparameters
input_size = len(selected_features)  # Number of features
hidden_size = 64
output_size = 1  # Predicting pressure (binary or continuous)
num_layers = 2
learning_rate = 0.001
num_epochs = 20

# Instantiate the model
model = PlayRNN(input_size, hidden_size, output_size, num_layers)
criterion = nn.BCEWithLogitsLoss()  # Binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:  # Assuming a DataLoader setup
        # Assuming inputs are already padded
        lengths = [len(seq) for seq in inputs]  # Get sequence lengths for packing
        packed_input = pack_padded_sequence(inputs, lengths, batch_first=True, enforce_sorted=False)

        # Forward pass
        outputs = model(packed_input)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


ValueError: too many values to unpack (expected 2)